In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface-hub
!pip install -q safetensors

##Hugging Face'e giriş

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

##Model ve tokenizer'ı yükleme

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

##Veri Setinin Hazırlanması

In [ ]:
python_explain_data = [
    {
        "code": """def add(a, b):
          return a + b""",
        "explanation": "Bu fonksiyon iki parametre alır (a ve b) ve ikisini toplayarak sonucu döndürür."
    },
    {
        "code": """numbers = [1, 2, 3, 4, 5]
          total = 0
          for n in numbers:
           total += n
          print(total)""",
        "explanation": "Önce 1'den 5'e kadar sayılardan oluşan bir liste tanımlanıyor. 'total' değişkeni 0 olarak başlatılıyor. for döngüsüyle listedeki her sayı 'total' üzerine ekleniyor. Son olarak toplam ekrana yazdırılıyor."
    },
    {
        "code": """for i in range(5):
          print("Hello", i)""",
        "explanation": "range(5) ifadesi 0'dan 4'e kadar sayılar üretir. Döngü her adımda 'Hello' ve o anki 'i' değerini ekrana yazdırır."
    },
    {
        "code": """def is_even(n):
          if n % 2 == 0:
            return True
          else:
            return False""",
        "explanation": "Bu fonksiyon n sayısının 2 ile bölümünden kalanı kontrol eder. Kalan 0 ise sayı çifttir ve True döndürülür, değilse False döndürülür."
    },
    {
        "code": """try:
          x = int(input("Sayı gir: "))
          print(10 / x)
          except ZeroDivisionError:
          print("Sıfıra bölemezsin!")
""",
        "explanation": "Kullanıcıdan bir sayı alınıp tamsayıya çevrilir. Ardından 10/x hesaplanır. Eğer kullanıcı 0 girerse ZeroDivisionError hatası oluşur ve except bloğu çalışarak uyarı mesajı yazdırılır."
    },
]


##HuggingFace Dataset nesnesi oluşturma

In [ ]:
from datasets import Dataset

def make_example(example):
    instruction = (
        "Türkçe ve anlaşılır bir şekilde, aşağıdaki Python kodunu satır satır açıkla:\n"
        f"{example['code']}"
    )
    target = example["explanation"]
    return {"input_text": instruction, "target_text": target}

converted = [make_example(e) for e in python_explain_data] #Listedeki her elemanı birer örneğe dönüştürüyoruz
dataset = Dataset.from_list(converted) #Listeyi HuggingFace dataset formatında dönüştürüyoruz
dataset = dataset.train_test_split(test_size=0.2, seed=42) #Veri setini dağıtıyoruz train %80 test %20

In [ ]:
print(dataset)

In [ ]:
dataset["train"][0]

##Tokenization ayarları

In [ ]:
max_input_length =  #girdi
max_target_length = 128 #çıktı

def preprocess(batch):
    # Encoder
    model_inputs = tokenizer(
        batch["input_text"],
        max_length=max_input_length,
        truncation=True,        # max_input_length'i aşarsa kes
        padding="max_length",   # max_input_length'e kadar padding yap
    )

    # Decoder
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target_text"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length",
        )

    #doğru cevaplar temsil ediliyor
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Preprocess fonktiyonları uygulanır ve input/target kolonları kaldırılır

In [ ]:
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input_text", "target_text"])

In [ ]:
tokenized_dataset

##Data collator ve eğitim argümanları

"evaluate" ekleyip metrik karşılaşması yapılabilir

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #dinamik padding ile model beslenir

training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-python-explainer",
    eval_strategy="steps",
    eval_steps=20,
    logging_steps=10,
    save_steps=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=100,             #Toplam epoch sayısı
    learning_rate=5e-5,               #Öğrenme oranı
    predict_with_generate=True,
    fp16=False,
    save_total_limit=1,
    load_best_model_at_end=False,
)

trainer = Seq2SeqTrainer(
    model=model,                                # Eğitilecek model
    args=training_args,                         # Eğitim ayarları
    train_dataset=tokenized_dataset["train"],   # Eğitim verisi
    eval_dataset=tokenized_dataset["test"],     # Test verisi
    tokenizer=tokenizer,                        # Tokenizer
    data_collator=data_collator,
)


In [ ]:
trainer.train()

Modeli kaydetme

In [ ]:
trainer.save_model("flan-t5-python-explainer")
tokenizer.save_pretrained("flan-t5-python-explainer")

##Yeni kodları açıklayan fonksiyon tanımlama

Bu fonksiyon, verilen Python kodu için Türkçe açıklama üretir.

In [ ]:
def explain_code(code: str, max_new_tokens: int = 128) -> str:
    instruction = (
        "Türkçe ve anlaşılır bir şekilde, aşağıdaki Python kodunu satır satır açıkla:\n"
        f"{code}"
    )

    # Instructionı token'lara çeviriyoruz.
    inputs = tokenizer(
        instruction,
        return_tensors="pt",          # PyTorch tensör formatında döndür
        truncation=True,
        max_length=max_input_length,
    ).to(model.device)                # Modelin çalıştığı cihaza (CPU/GPU) gönder

    #Modelin çıktısı
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,          # Uygunsa modeli durdur
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

##Test

In [ ]:
test_code = """my_list = [10, 20, 30]
for i, val in enumerate(my_list):
    print(i, val)"""

print(explain_code(test_code))

##HuggingFace

In [ ]:
!zip -r flan-t5-python-explainer.zip flan-t5-python-explainer

In [ ]:
from google.colab import files
files.download("flan-t5-python-explainer.zip")